In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'heat stress guidlines interventions mitigation strategies' 
}

In [ ]:
##GET TITLE/LINK/AUTHORS OF N*10 ARTICLES

#loop through pages for n pages
n = 10

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns=['title', 'link', 'author'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'title': titles, 'link': links, 'author': authors}))

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
results.to_csv('article_info.csv')

In [4]:
## BASIC CODE TO SCRAPE ABSTRACT 

import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [5]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [400]>


In [6]:
## CODE TO DOWNLOAD 1 LINK

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [7]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
import os
import random

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome()  # initialize web driver
    main_page_url = 'https://sci-hub.ru/'  # open sci-hub
    driver.get(main_page_url)

    try:
        # Wait until the search bar with id="request" is present
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'request'))
        )
        search_bar.send_keys(url_to_search)
        search_bar.send_keys(Keys.RETURN)  # Submit the search query
    except Exception as e:
        print(f"Could not find search bar for {url_to_search}. Error: {e}")
        driver.quit()
        return None

    time.sleep(random.uniform(4, 6))

    try:
        # Find download button
        download_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[@onclick]"))
        )
        onclick_attribute = download_button.get_attribute('onclick')
    except Exception as e:
        print(f"Could not find download button for {url_to_search}. Error: {e}")
        driver.quit()
        return url_to_search  # Mark this URL as problematic

    try:
        # Extract URL from onclick attribute
        pdf_url = onclick_attribute.split("'")[1]
        download_link = 'https://sci-hub.ru' + pdf_url

        # Ensure 'pdfs' folder exists
        os.makedirs('pdfs', exist_ok=True)

        # Download and save the PDF
        response = requests.get(download_link)
        with open(f'pdfs/{file_name}.pdf', 'wb') as file:
            file.write(response.content)

    except Exception as e:
        print(f"Failed to download or save PDF for {url_to_search}. Error: {e}")
        driver.quit()
        return url_to_search

    driver.quit()
    return None  # Success


In [8]:
##to call on all links !!! DO NOT RERUN, IT WILL SET LIST BACK TO NONE

import pandas as pd
import os

data = pd.read_csv('article_info.csv')
title = data['title']
links = data['link']
author = ['author'] 


def file_exists(file_path):
    return os.path.exists(file_path)

list_error_links = []

In [9]:
#function to call on all links
os.makedirs('pdfs', exist_ok=True)

for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'
    if file_exists(pdf_file_path) is True:
        print("PDF already downloaded")
    else:
        if row['link'] in list_error_links:
            print('error already encountered')
        else:
            url_returned = pdf_download(row['link'], index)  #(links[index], index)
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'ERROR FOR {url_returned}')

PDF already downloaded
Could not find download button for https://www.jstage.jst.go.jp/article/indhealth/51/1/51_2012-0089/_article/-char/ja/. Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7D79F4C25+3179557]
	(No symbol) [0x00007FF7D76588A0]
	(No symbol) [0x00007FF7D74E91CA]
	(No symbol) [0x00007FF7D753FA67]
	(No symbol) [0x00007FF7D753FC9C]
	(No symbol) [0x00007FF7D7593627]
	(No symbol) [0x00007FF7D7567C6F]
	(No symbol) [0x00007FF7D75902F3]
	(No symbol) [0x00007FF7D7567A03]
	(No symbol) [0x00007FF7D75306D0]
	(No symbol) [0x00007FF7D7531983]
	GetHandleVerifier [0x00007FF7D7A567CD+3579853]
	GetHandleVerifier [0x00007FF7D7A6D1D2+3672530]
	GetHandleVerifier [0x00007FF7D7A62153+3627347]
	GetHandleVerifier [0x00007FF7D77C092A+868650]
	(No symbol) [0x00007FF7D7662FFF]
	(No symbol) [0x00007FF7D765F4A4]
	(No symbol) [0x00007FF7D765F646]
	(No symbol) [0x00007FF7D764EAA9]
	BaseThreadInitThunk [0x00007FFDCBF8E8D7+23]
	RtlUserThreadStart [0x00007FFDCCEDBF6C+44]

ERROR FOR https://www.j

In [10]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pypdf import PdfReader
from os import listdir
from os.path import isfile, join

mypath = 'pdfs'  # or your full path
pdf_folder_content = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.lower().endswith('.pdf')]

all_pdf_texts = {}

for pdf in pdf_folder_content:
    print(f'Analyzing {pdf}...')

    try:
        reader = PdfReader(join(mypath, pdf))
        n_pages = len(reader.pages)
        print(f'{pdf} has {n_pages} pages!')

        all_text = []
        for i in range(n_pages):  
            page_text = reader.pages[i].extract_text()
            all_text.append(page_text)

        result_string = ' '.join(filter(None, all_text))  # remove None entries
        all_pdf_texts[pdf] = result_string

    except Exception as e:
        print(f"Skipping {pdf} — couldn't read PDF. Error: {e}")


Analyzing 0.pdf...
0.pdf has 16 pages!


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Analyzing 2.pdf...
Skipping 2.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 3.pdf...
Skipping 3.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 5.pdf...
Skipping 5.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 6.pdf...
6.pdf has 23 pages!
Analyzing 7.pdf...
7.pdf has 11 pages!


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Analyzing 8.pdf...
Skipping 8.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 9.pdf...
Skipping 9.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
